<a href="https://colab.research.google.com/github/akshay-akm/codsoft/blob/main/Copy_of_numerical_data_bad_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2023 Google LLC. Double-click for license information.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [ ]:
df = pd.read_csv('/content/spam.csv', encoding='latin-1')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df.duplicated().sum())
print(df.isnull().sum())
print(df.info())

In [ ]:
df.rename(columns={'v1':'target','v2':'texts' }, inplace=True)
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace=True)

In [ ]:
mp = {'spam':0,
      'ham':1}
df['target'] =  df['target'].map(mp)

In [ ]:
df = df.drop_duplicates(keep='first')

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
df['text_char'] = df['texts'].apply(len)
df['word_num'] = df['texts'].apply(lambda x : len(nltk.word_tokenize(x)))
df['sent_num'] = df['texts'].apply(lambda x : len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string
from nltk.stem.porter import PorterStemmer

pc = PorterStemmer()

In [ ]:
def text_process(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y= []
    for i in text :
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text :
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()
    for i in text:
        y.append(pc.stem(i))
    return " ".join(y)

In [ ]:
text_process('I am testing This as the function is working correctly or not , TesT %??')

In [ ]:
df['Transformed_text'] = df['texts'].apply(text_process)

In [ ]:
df.head(1)

In [ ]:
df[df['target'] == 0].describe().T

In [ ]:
df[df['target'] == 1].describe().T

In [ ]:
df['target'].value_counts().plot.pie(autopct = "%1.1f%%", explode = [0.25, 0], shadow = True)

In [ ]:
df.head(2)

In [ ]:
eda = [
    'text_char', 'word_num', 'sent_num'
]
for i in eda:
    plt.figure(figsize=(10,4))
    sns.histplot(data = df , x= i, palette='viridis', hue="target")
    plt.title(f'Hist plot of {i}')
    plt.tight_layout()
    plt.show()

In [ ]:
df.corr(numeric_only=True)

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize=(8,6))
wc = WordCloud(width = 1000, height = 1000, min_font_size = 12, background_color = 'white')
figu = wc.generate(df[df['target']== 0]['Transformed_text'].str.cat(sep=' '))
plt.tight_layout()
plt.imshow(figu)

In [ ]:
plt.figure(figsize=(8,6))
wc = WordCloud(width = 1000, height = 1000, min_font_size = 12, background_color = 'white')
figu = wc.generate(df[df['target']== 1]['Transformed_text'].str.cat(sep=' '))
plt.tight_layout()
plt.imshow(figu)

In [ ]:
spam_a = []
for msg in df[df['target']==0]['Transformed_text'].tolist():
    for word in msg.split():
        spam_a.append(word)

In [ ]:
from collections import Counter
plt.figure(figsize=(8,6))
sns.barplot(x = pd.DataFrame(Counter(spam_a).most_common(25))[0],y= pd.DataFrame(Counter(spam_a).most_common(25))[1])
plt.tight_layout()
plt.xticks(rotation = 90)
plt.show()

In [ ]:
ham_a = []
for msg in df[df['target']==1]['Transformed_text'].tolist():
    for word in msg.split():
        ham_a.append(word)
plt.figure(figsize=(8,6))
sns.barplot(x = pd.DataFrame(Counter(ham_a).most_common(25))[0],y= pd.DataFrame(Counter(ham_a).most_common(25))[1])
plt.tight_layout()
plt.xticks(rotation = 90)
plt.show()